In [218]:
import os
import pandas as pd
from pprint import pprint
from collections import OrderedDict
import warnings

warnings.filterwarnings("ignore")

In [315]:
data_df = pd.read_excel("uart_reg.xlsx", encoding="utf-8")
data_df.fillna("", inplace=True)
data_df.head(12)

,#Name,Addr,Area,Signal_name,OP,ResetValue,Autoclear,Clockdomain
0,UARTDR,0x000,[15:12],reserved,RO,0x0,,
1,,,[11:11],OE,RW,0x0,,
2,,,[10:10],BE,RW,0x0,,
3,,,[9:9],PE,RW,0x0,,
4,,,[8:8],FE,RW,0x0,,
5,,,[7:0],DATA,RW,0x0,,
6,UARTRSR,0x004,[15:4],reserved,RO,0x0,,
7,,,[3:3],OE,RW,0x0,,
8,,,[2:2],BE,RW,0x0,,
9,,,[1:1],PE,RW,0x0,,


In [221]:
def get_field_info(info):
    """
    get filed info to list
    """
    index_e, index_s = info[0][1:-1].split(":")
    index_e, index_s = int(index_e), int(index_s)
    filed_li = [info[1], str(index_e-index_s+1), str(index_s), "\"" + info[2] + "\"" , "0", "'h" + format(eval(info[3]), 'X'), "1", "1", "1"]
    return filed_li

In [314]:
# get all info to orderdict
reg_info = OrderedDict()
for li in data_df.values.tolist():
    if li[0]:
        reg_name = li[0]
        reg_info[reg_name] = OrderedDict({"name":li[0], "addr":li[1], "field":[]})
        reg_info[reg_name]["field"].append(get_field_info(li[2:]))
    else:
        reg_info[reg_name]["field"].append(get_field_info(li[2:]))
        
reg_info

OrderedDict([('UARTDR',
              OrderedDict([('name', 'UARTDR'),
                           ('addr', '0x000'),
                           ('field',
                            [['reserved',
                              '4',
                              '12',
                              '"RO"',
                              '0',
                              "'h0",
                              '1',
                              '1',
                              '1'],
                             ['OE',
                              '1',
                              '11',
                              '"RW"',
                              '0',
                              "'h0",
                              '1',
                              '1',
                              '1'],
                             ['BE',
                              '1',
                              '10',
                              '"RW"',
                              '0',
              

In [316]:
# reg_info['UARTDR']['field']

In [317]:
# the start string of object
start_info="uart"

In [224]:
# normal code of register class
reg_class_info = [
    "class %s extends uvm_reg;",
    "    `uvm_object_utils(%s)",
    "    virtual function void build();",
    "        %s = uvm_reg_field::type_id::create(\"%s\");",
    "        %s.configure(this, %s);",
    "    function new(string name = \"%s\");",
    "    endfunction",
    "        super.new(name, 32, UVM_NO_COVERAGE);"
]

In [255]:
# get one register class code
def gen_reg_one(reg_name, start_info):
    reg_name_full = "%s_reg_%s" % (start_info, reg_name)
    reg_class_sv_all = []

    reg_class_sv_all.append(reg_class_info[0] % reg_name_full)
    reg_class_sv_all.append(reg_class_info[1] % reg_name_full)
    reg_class_sv_all.append("")

    
    filed_rand = [] # get the all fileds 
    filed_cg = [] # create and configure

    for li in reg_info[reg_name]['field']:
        filed_rand.append("    rand uvm_reg_field %s;" % li[0])

        filed_cg.append(reg_class_info[3] % (li[0], li[0]))  # create
        filed_cg.append(reg_class_info[4] % (li[0], ", ".join(li[1:])))  # configure
        filed_cg.append("")

    reg_class_sv_all.extend(filed_rand)  # extend register rand codes
    reg_class_sv_all.append("")
    
    reg_class_sv_all.append(reg_class_info[2])
    
    reg_class_sv_all.extend(filed_cg) # extend register create and configure codes
    
    reg_class_sv_all.append(reg_class_info[6]) # endfunction
    reg_class_sv_all.append("")
    
    # the new function
    reg_class_sv_all.append(reg_class_info[5] % reg_name_full)
    reg_class_sv_all.append(reg_class_info[7])
    reg_class_sv_all.append(reg_class_info[6])
    
    reg_class_sv_all.append("endclass")
    reg_class_sv_all.append("")
    return reg_class_sv_all

In [286]:
gen_reg_all_sv = []  # all register model codes

for reg_name in reg_info.keys():
    print(reg_name)
    reg_sv_one = gen_reg_one(reg_name, start_info=start_info)
    gen_reg_all_sv.extend(reg_sv_one)

UARTDR
UARTRSR
UARTFR
UARTIBRD
UARTFBRD
UARTLLCR_H
UARTCR
UARTIFLS
UARTIMSC
UARTRIS
UARTMIS
UARTICR


In [300]:
reg_block_info = [
    "class %s extends uvm_reg_block;",
    "    rand %s_reg_%s %s;",
    "    `uvm_object_utils(%s)",
    "    function new(string name = \"%s\");",
    "        super.new(name, UVM_NO_COVERAGE);",
    "    endfunction",
    "    virtual function void build();",
    "        default_map = create_map(\"default_map\", 0, 4, UVM_BIG_ENDIAN, 1);",
    "        %s = %s::type_id::create(\"%s\", null, get_full_name());",
    "        %s.configure(this, null, \"\");",
    "        %s.build();",
    "        default_map.add_reg(%s, %s, %s);"
]

In [306]:
reg_block_name = "ral_%s_reg" % start_info

In [307]:
reg_block_sv_all = []  # all code of register block
reg_block_sv_all.append(reg_block_info[0] % reg_block_name)  # define register block class
reg_block_sv_all.append("")

# the rand code of all reigister
rand_reg_li = []
for reg_name in reg_info.keys():
    rand_reg_li.append(reg_block_info[1] % (start_info, reg_name, reg_name))
    
reg_block_sv_all.extend(rand_reg_li) # extend
reg_block_sv_all.append("")

reg_block_sv_all.append(reg_block_info[2] % reg_block_name)
reg_block_sv_all.append("")

# the new function
reg_block_sv_all.append(reg_block_info[3] % reg_block_name)
reg_block_sv_all.append(reg_block_info[4])
reg_block_sv_all.append(reg_block_info[5])
reg_block_sv_all.append("")

reg_block_sv_all.append(reg_block_info[6]) # the build function
reg_block_sv_all.append(reg_block_info[7]) # create_map
reg_block_sv_all.append("")

In [310]:
# all code of add all register
b_gen_reg_all = []
for reg_name in reg_info.keys():
    print(reg_name)
    b_gen_reg_one = []
    
    b_gen_reg_one.append(reg_block_info[8] % (reg_name, start_info + "_reg_" + reg_name, reg_name))
    b_gen_reg_one.append(reg_block_info[9] % reg_name)
    b_gen_reg_one.append(reg_block_info[10] % reg_name)
    b_gen_reg_one.append(reg_block_info[11] % (reg_name, "'h0" + format(eval(reg_info[reg_name]["addr"]), 'X'), "\"RW\""))
    b_gen_reg_one.append("")
    
    b_gen_reg_all.extend(b_gen_reg_one)
    
reg_block_sv_all.extend(b_gen_reg_all)
reg_block_sv_all.append(reg_block_info[5])
reg_block_sv_all.append("endclass")

UARTDR
UARTRSR
UARTFR
UARTIBRD
UARTFBRD
UARTLLCR_H
UARTCR
UARTIFLS
UARTIMSC
UARTRIS
UARTMIS
UARTICR


In [312]:
gen_reg_all_sv.extend(reg_block_sv_all)

In [313]:
for line in gen_reg_all_sv:
    print(line)

class uart_reg_UARTDR extends uvm_reg;
    `uvm_object_utils(uart_reg_UARTDR)

    rand uvm_reg_field reserved;
    rand uvm_reg_field OE;
    rand uvm_reg_field BE;
    rand uvm_reg_field PE;
    rand uvm_reg_field FE;
    rand uvm_reg_field DATA;

    virtual function void build();
        reserved = uvm_reg_field::type_id::create("reserved");
        reserved.configure(this, 4, 12, "RO", 0, 'h0, 1, 1, 1);

        OE = uvm_reg_field::type_id::create("OE");
        OE.configure(this, 1, 11, "RW", 0, 'h0, 1, 1, 1);

        BE = uvm_reg_field::type_id::create("BE");
        BE.configure(this, 1, 10, "RW", 0, 'h0, 1, 1, 1);

        PE = uvm_reg_field::type_id::create("PE");
        PE.configure(this, 1, 9, "RW", 0, 'h0, 1, 1, 1);

        FE = uvm_reg_field::type_id::create("FE");
        FE.configure(this, 1, 8, "RW", 0, 'h0, 1, 1, 1);

        DATA = uvm_reg_field::type_id::create("DATA");
        DATA.configure(this, 8, 0, "RW", 0, 'h0, 1, 1, 1);

    endfunction

    function new(

In [322]:
with open("ral_" + start_info + "_regmodel.sv", "w", encoding="utf-8") as f:
    for line in gen_reg_all_sv:
        f.write(line + "\n")

39

39

1

33

27

...

25

51

1

16

9

In [239]:
import re
import csv
import os
# from rkvRgmItem import RgmItem


class RgmItem:
    '''
    classdocs
    '''

    def __init__(self):
        '''
        Constructor
        '''
        self._properties = {'register': '', 'address': '0x00', 'fields': []
                            }


class GenRgm:
    '''
    classdocs
    '''

    def __init__(self):
        '''
        Constructor
        '''
        self._rgmItemList = []
        self._curRegName = ''
        self._uvmFileName = 'rkv_watchdog_reg.sv'
        self._cFileName = 'rkv_watchdog_reg.h'

    # Excel/CSV format file read/write methods

    def readRgmFile(self, filename):
        "read register file and update the internal database"
        rgmFile = open(filename, 'r')
        self._rgmFileName = filename
        reader = csv.DictReader(rgmFile)
        for infos in reader:
            self._readRgmInfos(infos)

    def _readRgmInfos(self, infos):
        """extract register information from string line
           and return it as a dictionary
        """
        if(infos['register'] != ''):
            rgmObj = RgmItem()
            self._curRegName = infos['register']
            rgmObj._properties['register'] = infos['register']
            rgmObj._properties['address'] = infos['address']
            self._rgmItemList.append(rgmObj)
            
        fieldInfos= self._readFieldInfos(infos)
        self._rgmItemList[-1]._properties['fields'].append(fieldInfos)


    def _readFieldInfos(self, infos):
        fieldInfos= {}
        fieldInfos['field']= infos['field']
        fieldInfos['field_access']= infos['field_access']
        fieldInfos['reset_value']= infos['reset_value']
        fieldInfos['bitpos_end']= infos['bitpos_end']
        fieldInfos['bitpos_start']= infos['bitpos_start']
        fieldInfos['function']= infos['function']
        return fieldInfos


    def genUVMFile(self):
        with open(self._uvmFileName, 'w') as uvmFile:

            def _writeUVMHeader():
                # uvmFile.write('package rkv_watchdog_reg;\n')
                uvmFile.write('  import uvm_pkg::*;\n')
                uvmFile.write('  `include "uvm_macros.svh"\n')
            def _writeRegClass(rgmObj):
                uvmFile.write('  class %s_reg extends uvm_reg;\n' % rgmObj._properties['register'])
                uvmFile.write('    `uvm_object_utils(%s_reg)\n' % rgmObj._properties['register'])
                for fd in rgmObj._properties['fields']:
                    uvmFile.write('    rand uvm_reg_field %s;\n' % fd['field'])
                uvmFile.write('    covergroup value_cg;\n')
                uvmFile.write('      option.per_instance = 1;\n')
                for fd in rgmObj._properties['fields']:
                    uvmFile.write('      %s: coverpoint %s.value[%s:%s];\n' % (fd['field'], fd['field'], fd['bitpos_end'], fd['bitpos_start']))
                uvmFile.write('    endgroup\n')
                uvmFile.write('\
    function new(string name = "%s_reg");\n\
      super.new(name, 32, UVM_CVR_ALL);\n\
      void\'(set_coverage(UVM_CVR_FIELD_VALS));\n\
      if(has_coverage(UVM_CVR_FIELD_VALS)) begin\n\
        value_cg = new();\n\
      end\n\
    endfunction\n' % rgmObj._properties['register'])
                uvmFile.write('    virtual function void build();\n')
                for fd in rgmObj._properties['fields']:
                    uvmFile.write('      %s = uvm_reg_field::type_id::create("%s");\n' % (fd['field'], fd['field']))
                for fd in rgmObj._properties['fields']:
                    uvmFile.write('      %s.configure(this, %0d, %s, "%s", 0, \'h%s, 1, 0, 0);\n' % (fd['field'], int(fd['bitpos_end'])-int(fd['bitpos_start'])+1, fd['bitpos_start'], fd['field_access'], fd['reset_value'].replace('0x', '')))
                uvmFile.write('    endfunction\n')
                uvmFile.write('\
    function void sample(\n\
      uvm_reg_data_t data,\n\
      uvm_reg_data_t byte_en,\n\
      bit            is_read,\n\
      uvm_reg_map    map\n\
    );\n\
      super.sample(data, byte_en, is_read, map);\n\
      sample_values(); \n\
    endfunction\n\
    function void sample_values();\n\
      super.sample_values();\n\
      if (get_coverage(UVM_CVR_FIELD_VALS)) begin\n\
        value_cg.sample();\n\
      end\n\
    endfunction\n\
  endclass\n\n')
                
            def _writeRgmClass():
                uvmFile.write('  class rkv_watchdog_rgm extends uvm_reg_block;\n')
                uvmFile.write('    `uvm_object_utils(rkv_watchdog_rgm)\n')
                for rg in self._rgmItemList:
                    uvmFile.write('    rand %s_reg %s;\n' % (rg._properties['register'], rg._properties['register']))



def testGenUVMRgm():
    print('%s \n[testReadClockFile] started \n%s\n' % (40*'*', 40*'*'))
    gen = GenRgm()
    gen.readRgmFile('rkv_watchdog_reg.csv')
    gen.genUVMFile()
    print('%s \n[testReadClockFile] PASSED \n%s\n' % (40*'*', 40*'*'))
moduleTestList.append(testReadRgmFile)

# testReadRgmFile()
testGenUVMRgm()

NameError: name 'moduleTestList' is not defined